In [1]:
# Import libraries.
import requests
from urllib.request import urlopen
from PIL import Image
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import re

Two types of api links are used for this work.
* Class ranking API: Contains top 200 players of the class with character information.
* Character Information API: Contains specific character information in structured format.

The API link for class ranking is as followed: <br>
https://cdn.dfcat.net/api/rank/{1}

Replace {1} with class second awakening name. See list separate ipynb.

<u>Male Slayer</u>
* swordgod
* darklord
* bloodevil
* indaratian
* rakshasa



The link for character info requires to be hashed. The hashed link is given via the following link: <br>
https://api.dfcat.net/api/character/info?serverId={1}&characterId={2}

Replace {1} with serverId, {2} with characterId.

__Work showcase for witch/geniewiz__

In [2]:
# Function for getting character general information.
def get_char_general_info(char_info_data):

    general_info = []

    # Get character general information and append to appropriate lists.
    general_info.append(char_info_data["serverId"])
    general_info.append(char_info_data["characterId"])
    general_info.append(char_info_data["adventureName"])
    general_info.append(char_info_data["characterName"])
    general_info.append(char_info_data["job"])
    general_info.append(char_info_data["guildName"])
    general_info.append(char_info_data["summary"]["main"])
    general_info.append(char_info_data["summary"]["status"][1]["value"])

    return general_info

In [3]:
# Function for getting character equipment itemId (for image link)
def get_char_eq_info(equipment_data):

    # Get character equipment itemId (for image link) and append to appropriate lists.
    # For each equipment and equipment upgrade (if applicable), check if they exists (by looking if "itemId" key exists in the appropriate levels).
    # If there is no ungrade for the equipment slot yet, put NaN.
    # Title, creature and insignia do not have "upgradeInfo" key.

    # Define an empty list.
    eq_info = []

    # Loop thru each equipment item.
    for idx, slot in enumerate(equipment_data):

        # Get equipment info.
        if "itemId" in equipment_data[idx]:
            eq_info.append(equipment_data[idx]["itemId"])
        else:
            eq_info.append(np.nan)

        # idx condition to exclude title, creature and insignia.
        if idx < 13:

            # Get equipment upgrade info.
            if "upgradeInfo" in equipment_data[idx]:
                eq_info.append(equipment_data[idx]["upgradeInfo"]["itemId"])
            else:
                eq_info.append(np.nan)

    return eq_info

In [4]:
# Function for getting character talisman itemId (for skill icon image link)
def get_char_talisman_info(talisman_data):

    # Define an empty list.
    talisman_info = []

    # Loop thru each talisman item.
    for idx, slot in enumerate(talisman_data):

        # Get talisman info.
        if "itemId" in talisman_data[idx]:
            talisman_info.append(talisman_data[idx]["itemId"])
            talisman_info.append(talisman_data[idx]["icon"])
        else:
            talisman_info.append(np.nan)
            talisman_info.append(np.nan)

    return talisman_info

In [13]:
# Get ranking data for witch class.
# Parse data as json to segregate data by characters.
class_rank_data = requests.get("https://cdn.dfcat.net/api/rank/geniewiz").json()

# Pre-define columns for dataframe.
col = ["serverId", "characterId", "adventureName", "characterName", "job", "guildName", "summary", "fame",
        "weapon_id", "weapon_upgrade_id",
        "weapon2_id", "weapon2_upgrade_id",
        "top_id", "top_upgrade_id",
        "bottom_id", "bottom_upgrade_id",
        "shoulder_id", "shoulder_upgrade_id",
        "belt_id", "belt_upgrade_id",
        "shoes_id", "shoes_upgrade_id",
        "bracelet_id", "bracelet_upgrade_id",
        "necklace_id", "necklace_upgrade_id",
        "ring_id", "ring_upgrade_id",
        "subeq_id", "subeq_upgrade_id",
        "magicstone_id", "magicstone_upgrade_id",
        "earring_id", "earring_upgrade_id",
        "title_id", "creature_id","insignia_id",
        "talisman1_id", "talisman1_skill_icon", "talisman2_id", "talisman2_skill_icon", "talisman3_id", "talisman3_skill_icon"]

# Create a data frame for class ranking.
class_rank_df = pd.DataFrame(columns=col)

# Loop thru each character in class_rank_data
for each_char in class_rank_data["rank"]:

    # Define an empty list.
    char_info_list = []
    
    # Replace all double quotes in each_char with empty string.
    each_char = each_char.replace("\"", "")

    # Find the serverId, then retrieve it from regex group 1.
    serveId_find = re.search("serverId:([a-zA-Z]+),", each_char)
    serveId = serveId_find.group(1)

    # Find the characterId, then retrieve it from regex group 1.
    characterId_find = re.search("characterId:([a-zA-Z0-9]+),", each_char)
    characterId = characterId_find.group(1)

    # Prepare the charcter information hashed url.
    char_info_url = "https://api.dfcat.net/api/character/info?serverId={}&characterId={}".format(serveId, characterId)

    # Get hashed url content, parse data as json and retrieve the character information hashed url under the key "redirect".
    char_info_hashed_url = requests.get(char_info_url).json()["redirect"]

    # Get character information content, parse data as json.
    char_info_data = requests.get(char_info_hashed_url).json()


    # Get character general information and append to primary list.
    char_info_list = char_info_list + get_char_general_info(char_info_data)


    # Create data variable for equipment.
    equipment_data = char_info_data["info"]["equipment"]
    # Get character equipment information and append to primary list.
    char_info_list = char_info_list + get_char_eq_info(equipment_data)
    

    # Create data variable for talisman.
    talisman_data = char_info_data["info"]["talismans"]
    # Get character talisman information and append to primary list.
    char_info_list = char_info_list + get_char_talisman_info(talisman_data)


    # Insert list contents into new row of class_rank_df.
    class_rank_df.loc[len(class_rank_df)] = char_info_list

#### __Class Character Rank Summary__

The API link for item image is as followed: <br>
https://img-api.neople.co.kr/df/items/{1}

Replace {1} with item id.

<br>

The API link for character image is as followed: <br>
https://img-api.neople.co.kr/df/servers/{1}/characters/{2}

Replace {1} with server id and {2} with character id.

In [14]:
# html table text template for each class ranking of character.

hmtl_table_char_summary = """  
<table style=\"border: 1px solid black\">
    <tr>                                <!-- Table first row -->

        <td>#{}<td>                     <!-- Rank number -->
        <td rowspan=\"2\">{}</td>       <!-- Character image -->
        <td>{}<br>{}<td>                <!-- Explorer club name, Character name -->

        <td>{}</td>                     <!-- Weapon image -->
        <td>{}</td>                     <!-- Top image -->
        <td>{}</td>                     <!-- Bottom image -->
        <td>{}</td>                     <!-- Shoulder image -->
        <td>{}</td>                     <!-- Belt image -->
        <td>{}</td>                     <!-- Shoes image -->

        <td>{}</td>                     <!-- Bracelet image -->
        <td>{}</td>                     <!-- Necklace image -->
        <td>{}</td>                     <!-- Ring image -->    

        <td>{}</td>                     <!-- Sub eq image -->
        <td>{}</td>                     <!-- Magic stone image -->
        <td>{}</td>                     <!-- Earring image -->   

        <td>{}</td>                     <!-- Character summary -->

    </tr>

    <tr>                               <!-- Table second row -->

        <td>{}</td>                    <!-- Fame -->
        <td></td>                      <!-- Blank cell -->
        <td>{}<br>{}<td>               <!-- job, Guild name -->

        <td>{}</td>                    <!-- Weapon upgrade image -->
        <td>{}</td>                    <!-- Top upgrade image -->
        <td>{}</td>                    <!-- Bottom upgrade image -->
        <td>{}</td>                    <!-- Shoulder upgrade image -->
        <td>{}</td>                    <!-- Belt upgrade image -->
        <td>{}</td>                    <!-- Shoes upgrade image -->

        <td>{}</td>                     <!-- Bracelet upgrade image -->
        <td>{}</td>                     <!-- Necklace upgrade image -->
        <td>{}</td>                     <!-- Ring upgrade image -->    

        <td>{}</td>                     <!-- Sub eq upgrade image -->
        <td>{}</td>                     <!-- Magic stone upgrade image -->
        <td>{}</td>                     <!-- Earring upgrade image -->   

    </tr>

</table>
<br>
<br>
"""

In [17]:
# Define standard links.
item_img_std_link = "<img src=\"https://img-api.neople.co.kr/df/items/{}\">"
char_img_std_link = "<img src=\"https://img-api.neople.co.kr/df/servers/{}/characters/{}\" {}>"

# Define crop style for character image.
# The image is cropped using -ve margins, then clipped with clip-path (50px diameter at position x:100px, y:130px) to get a circle image.
# Note that the y position needs to be changed due to some characters being taller in nature. Expected y position is about 100px-130px.
char_img_crop_style = "style=\"background-color: bisque; clip-path: circle(50px at 100px 130px); margin: -90px 0px -60px 0px;\""

# Define an empty string for concatenation of html table strings.
html_output_char_summary = ""

# Loop thru each row in class_rank_df to get all information for each character.
for row, col in class_rank_df.iterrows():

    # Use string format to insert appropriate text, then concatenate the html table text for every character.
    html_output_char_summary = html_output_char_summary + hmtl_table_char_summary.format(
        
        row + 1,
        char_img_std_link.format(col["serverId"], col["characterId"], char_img_crop_style),  
        col["adventureName"], col["characterName"],
        
        "" if pd.isna(col["weapon_id"]) else item_img_std_link.format(col["weapon_id"]),
        "" if pd.isna(col["top_id"]) else item_img_std_link.format(col["top_id"]),                               
        "" if pd.isna(col["bottom_id"]) else item_img_std_link.format(col["bottom_id"]),
        "" if pd.isna(col["shoulder_id"]) else item_img_std_link.format(col["shoulder_id"]),
        "" if pd.isna(col["belt_id"]) else item_img_std_link.format(col["belt_id"]),
        "" if pd.isna(col["shoes_id"]) else item_img_std_link.format(col["shoes_id"]),                 

        "" if pd.isna(col["bracelet_id"]) else item_img_std_link.format(col["bracelet_id"]),  
        "" if pd.isna(col["necklace_id"]) else item_img_std_link.format(col["necklace_id"]),  
        "" if pd.isna(col["ring_id"]) else item_img_std_link.format(col["ring_id"]),             

        "" if pd.isna(col["subeq_id"]) else item_img_std_link.format(col["subeq_id"]),  
        "" if pd.isna(col["magicstone_id"]) else item_img_std_link.format(col["magicstone_id"]),
        "" if pd.isna(col["earring_id"]) else item_img_std_link.format(col["earring_id"]),
                         
        col["summary"], col["fame"], col["job"], col["guildName"],
                                

        "" if pd.isna(col["weapon_upgrade_id"]) else item_img_std_link.format(col["weapon_upgrade_id"]),
        "" if pd.isna(col["top_upgrade_id"]) else item_img_std_link.format(col["top_upgrade_id"]),
        "" if pd.isna(col["bottom_upgrade_id"]) else item_img_std_link.format(col["bottom_upgrade_id"]),
        "" if pd.isna(col["shoulder_upgrade_id"]) else item_img_std_link.format(col["shoulder_upgrade_id"]),
        "" if pd.isna(col["belt_upgrade_id"]) else item_img_std_link.format(col["belt_upgrade_id"]),
        "" if pd.isna(col["shoes_upgrade_id"]) else item_img_std_link.format(col["shoes_upgrade_id"]),

        "" if pd.isna(col["bracelet_upgrade_id"]) else item_img_std_link.format(col["bracelet_upgrade_id"]),
        "" if pd.isna(col["necklace_upgrade_id"]) else item_img_std_link.format(col["necklace_upgrade_id"]),
        "" if pd.isna(col["ring_upgrade_id"]) else item_img_std_link.format(col["ring_upgrade_id"]),

        "" if pd.isna(col["subeq_upgrade_id"]) else item_img_std_link.format(col["subeq_upgrade_id"]),
        "" if pd.isna(col["magicstone_upgrade_id"]) else item_img_std_link.format(col["magicstone_upgrade_id"]),
        "" if pd.isna(col["earring_upgrade_id"]) else item_img_std_link.format(col["earring_upgrade_id"]),   

    )

In [18]:
# Create html file.
file_html = open("RankingSummary_Witch.html", "w", encoding="utf-8")

# Write content into html file.
file_html.write(html_output_char_summary)

# Close html file
file_html.close()

#### __Class Character Equipment Summary__

In [19]:
# html table text template for equipment statistics.
# Contains html table for 1 row only.

html_table_eq_summary= """
<tr>                               <!-- Table 1 row -->
    <td>{}</td>                    <!-- Item image -->
    <td>{}</td>                    <!-- Item count -->
</tr>
"""

In [20]:
# Column list for equipment statistics.
cols = ["weapon_id", "weapon_upgrade_id",
        "top_id", "top_upgrade_id",
        "bottom_id", "bottom_upgrade_id",
        "shoulder_id", "shoulder_upgrade_id",
        "belt_id", "belt_upgrade_id",
        "shoes_id", "shoes_upgrade_id",
        "bracelet_id", "bracelet_upgrade_id",
        "necklace_id", "necklace_upgrade_id",
        "ring_id", "ring_upgrade_id",
        "subeq_id", "subeq_upgrade_id",
        "magicstone_id", "magicstone_upgrade_id",
        "earring_id", "earring_upgrade_id",
        "title_id"]

# Final html text output for equipment statistics.
html_output_eq_summary2 = ""

# Loop thru the columns to create html table for each slot, showing item count of all unique equipments.
for col in cols:

    # Intermediate html text output for equipment statistics.
    # Resets for every table item slot.
    html_output_eq_summary = ""

    # Loop thru every item in the grouped dataframe.
    for item_id, count in class_rank_df.groupby(col)[col].count().items():

        # Use string format to insert appropriate text, then concatenate the html table rows for every unique equipment.
        html_output_eq_summary = html_output_eq_summary + html_table_eq_summary.format(
            item_img_std_link.format(item_id),
            count
            )

    # Use string format to insert appropriate text, then concatenate all tables for every item slot.
    html_output_eq_summary2 = html_output_eq_summary2 + """  
    <table style=\"border: 1px solid black\">
        <tr>                               <!-- Table first row for slot name -->
            <td colspan=\"2\">{}</td>                    <!-- Item slot name -->
        </tr>
     {}
    </table>
    <br>
    <br>
    """.format(col.rsplit("_", 1)[0], html_output_eq_summary)



# Create temporary dataframes for all 3 talisman slots.
talisman1_df = pd.DataFrame({"talisman_id": class_rank_df["talisman1_id"],
                            "talisman_skill_icon": class_rank_df["talisman1_skill_icon"]})

talisman2_df = pd.DataFrame({"talisman_id": class_rank_df["talisman2_id"],
                            "talisman_skill_icon": class_rank_df["talisman2_skill_icon"]})

talisman3_df = pd.DataFrame({"talisman_id": class_rank_df["talisman3_id"],
                            "talisman_skill_icon": class_rank_df["talisman3_skill_icon"]})

# Concatenate the temporary dataframes.
talisman_df = pd.concat([talisman1_df, talisman2_df, talisman3_df])

# Initialise html_output_eq_summary for reuse.
html_output_eq_summary = ""

# Loop thru every item in the grouped dataframe.
for skill_icon, count in talisman_df.groupby("talisman_skill_icon")["talisman_skill_icon"].count().items():

    # Use string format to insert appropriate text, then concatenate the html table rows for every unique talisman.
    html_output_eq_summary = html_output_eq_summary + html_table_eq_summary.format(
        "<img src=\"{}\">".format(skill_icon),
        count
        )

# Use string format to insert appropriate text, then concatenate the talisman table to previous equipment tables.
html_output_eq_summary2 = html_output_eq_summary2 + """  
<table style=\"border: 1px solid black\">
    <tr>                                        <!-- Table first row for slot name -->
        <td colspan=\"2\">Talisman</td>         <!-- Item slot name -->
    </tr>
    {}
</table>
<br>
<br>
""".format(html_output_eq_summary)

In [21]:
# Create html file.
file_html = open("EquipmentCountSummary_Witch.html", "w", encoding="utf-8")

# Write content into html file.
file_html.write(html_output_eq_summary2)

# Close html file
file_html.close()

#### __Character Details__ <br>

Using the following character for sample.
* serverId: cain
* characterId: ec362ecd3b3d2d9f213b5ea29dc72ab6

In [22]:
serveId = "cain"
characterId = "ec362ecd3b3d2d9f213b5ea29dc72ab6"

# Prepare the charcter information hashed url.
char_info_url = "https://api.dfcat.net/api/character/info?serverId={}&characterId={}".format(serveId, characterId)

# Get hashed url content, parse data as json and retrieve the character information hashed url under the key "redirect".
char_info_hashed_url = requests.get(char_info_url).json()["redirect"]

# Get character information content, parse data as json.
char_info_data = requests.get(char_info_hashed_url).json()

# Create data variable for equipment.
equipment_data = char_info_data["info"]["equipment"]

In [23]:
# Text template for html.

# Define html style text.
html_style = """
<style>
    .column_200px {float: left; width: 200px;}
    .column_400px {float: left; width: 400px;}
    .column_600px {float: left; width: 600px;}
    .row:after {content: ""; display: table; clear: both;}
</style>
"""

# Define html div text for item details.
html_div_item_details = """
<div class="row" style="padding: 10px 0px 10px 10px; border: 1px solid black">
    <div class="row">
        <div class="column_200px">
            {}
        </div>
        <div class="column_200px">
            {}
        </div>
        <div class="column_400px">
            {}
            <br>
            {}
        </div>
        <div class="column_400px">
            {}
        </div>
    </div>
    <br>
    <div class="row">
        <div class="column_600px">
            {}
        </div>
        <div class="column_600px">
            {}
        </div>
    </div>
</div>
"""

In [24]:
# Initialise html_final_item_details as empty string variable.
html_final_item_details = ""

# Loop thru each equipment item.
for idx, slot in enumerate(equipment_data):

    # Loop up to item 13 (earring).
    if idx < 13:

        # Check for equipment existence.
        if "itemId" in equipment_data[idx]:

            # Get equipment image, together with upgrade equipment if exists.
            item_img = item_img_std_link.format(equipment_data[idx]["itemId"]) + item_img_std_link.format(equipment_data[idx]["upgradeInfo"]["itemId"]) if "itemId" in equipment_data[idx]["upgradeInfo"] else ""

            # Get amplification/reinforcement/refinement values.
            amp_reinforce_refine_text = equipment_data[idx]["amplificationName"] + "+" + str(equipment_data[idx]["reinforce"]) + "/" + str(equipment_data[idx]["refine"])

            # Get equipment name and option levels summary.
            itemName_text = equipment_data[idx]["itemName"] + "<br>" + \
                            equipment_data[idx]["upgradeInfo"]["itemName"] if "itemId" in equipment_data[idx]["upgradeInfo"] else "" + "<br>"

            olvl_text = str(equipment_data[idx]["fame"]["total"]) + "&nbsp" * 4 + \
                        str(equipment_data[idx]["growInfo"]["options"][0]["level"]) + "&nbsp" * 4 + \
                        str(equipment_data[idx]["growInfo"]["options"][1]["level"]) + "&nbsp" * 4 + \
                        str(equipment_data[idx]["growInfo"]["options"][2]["level"]) + "&nbsp" * 4 + \
                        str(equipment_data[idx]["growInfo"]["options"][3]["level"]) + "&nbsp" * 4 + \
                        str(equipment_data[idx]["growInfo"]["total"]["level"])

            # Get enchantment data.
            enchant_text = ""
            enchant_data = equipment_data[idx]["enchant"]["status"]

            # Loop thru every enchantment and get enchantment name and value.
            for enc_idx, enc in enumerate(enchant_data):
                if enchant_text == "":
                    enchant_text = enchant_data[enc_idx]["name"] + "+" + str(enchant_data[enc_idx]["value"])
                else:
                    enchant_text = enchant_text + "<br>" + enchant_data[enc_idx]["name"] + "+" + str(enchant_data[enc_idx]["value"])

            # Get item details.

            # (item details) Get itemGradeName, itemRarity, itemTypeDetail, itemAvailableLevel, fame.
            item_details = equipment_data[idx]["itemGradeName"] + "<br>" + equipment_data[idx]["itemRarity"] + " " + equipment_data[idx]["itemTypeDetail"] + "<br>레벨 제한 " + str(equipment_data[idx]["itemAvailableLevel"]) + "<br>" + \
                            str(equipment_data[idx]["fame"]["total"]) + " (" + str(equipment_data[idx]["fame"]["base"]) + " + " + str(equipment_data[idx]["fame"]["grow"])  + ")"
        
            # (item details) Get item status
            status_data = equipment_data[idx]["itemStatus"]
            # Loop thru every item status and get status name and value.
            for status_idx, enc in enumerate(status_data):
                item_details = item_details + "<br>" + status_data[status_idx]["name"] + "+" + str(status_data[status_idx]["value"])

            # (item details) Get equipment explain.
            explain_data = equipment_data[idx]["explain"]
            # Loop thru every equipment explain and get explain.
            for explain_idx, enc in enumerate(explain_data):
                item_details = item_details + "<br>" + explain_data[explain_idx]
                
            # (item details) Combine enchant_text.
            item_details = item_details + "<br><br>&ltEnchantment&gt<br>" + enchant_text + "<br><br>&ltFusion Option&gt"

            # (item details) Check upgrade type and get the upgrade info.
            if "bakalInfo" in equipment_data[idx]:
                fusion_data = equipment_data[idx]["bakalInfo"]
            elif "machineRevolutionInfo" in equipment_data[idx]:
                fusion_data = equipment_data[idx]["machineRevolutionInfo"]
            elif "ispinsInfo" in equipment_data[idx]:
                fusion_data = equipment_data[idx]["ispinsInfo"]
            elif "dimensionCloisterInfo" in equipment_data[idx]:
                fusion_data = equipment_data[idx]["dimensionCloisterInfo"]
            else:
                fusion_data = ""

            # Check if upgrade exists (fusion_data not empty)
            if fusion_data != "":
                # Loop thru every options.
                for fusion1_idx, fusion1 in enumerate(fusion_data["options"]):
                    # Loop thru every explain.
                    for fusion2 in fusion_data["options"][fusion1_idx]["explain"]:
                        # (item details) Get upgrade/fusion explain.
                        item_details = item_details + "<br>" + fusion2["mono"]


            # Get equipment growInfo.
            option_data = equipment_data[idx]["growInfo"]

            # Get total level and damage values.
            option_details = "&lt성장 옵션&gt" + "<br>옵션 레벨 합 " + str(option_data["total"]["level"]) + "<br>성장 옵션 총 피해 증가 " + str(option_data["total"]["damage"])
            
            # Loop thru all options.
            for option_idx, option in enumerate(option_data["options"]):
                # Get option level, expRate, damage
                option_details = option_details + "<br><br>" + \
                                "{} 옵션 Lv ".format(option_idx + 1) + str(option_data["options"][option_idx]["level"])  + " (" + str(option_data["options"][option_idx]["expRate"]) + "%)<br>" + \
                                "피해 증가 " + str(option_data["options"][option_idx]["damage"])

                # Loop thru all explains.
                for option_explain_idx, option_explain in enumerate(option_data["options"][option_idx]["explain"]):
                    # Get explain of equipment option.
                    option_details = option_details + "<br>" + option_data["options"][option_idx]["explain"][option_explain_idx]                            

            # Combine all information of one equipment into html_div_item_details and append to html_final_item_details.
            html_final_item_details = html_final_item_details + html_div_item_details.format(item_img, amp_reinforce_refine_text, itemName_text, olvl_text, enchant_text, item_details, option_details)


In [25]:
# Get equipment info for title.

# Get title image.
item_img = item_img_std_link.format(equipment_data[13]["itemId"])

# Get reinforcement value.
reinforce_text = "+" + str(equipment_data[13]["reinforce"])

# Get item name and reinforcement values.
itemName_reinforce_text = equipment_data[13]["itemName"] + "<br>피해증가 +" + str(equipment_data[13]["reinforce"])

# get item details.
item_details = equipment_data[13]["itemRarity"] + " " + equipment_data[13]["itemTypeDetail"] + "<br>" + \
                str(equipment_data[13]["reinforce"]) + " 강화" + "<br>" + \
                "모험가 명성 " + str(equipment_data[13]["fame"]["total"]) + "<br>"

# (item details) get status data.
status_data = equipment_data[13]["itemStatus"]
# Loop thru every item status and get status name and value.
for status_idx, enc in enumerate(status_data):
    item_details = item_details + "<br>" + status_data[status_idx]["name"] + "+" + str(status_data[status_idx]["value"])

# (item details) get explain data.
explain_data = equipment_data[13]["explain"]
# Loop thru every equipment explain and get explain.
for explain_idx, enc in enumerate(explain_data):
    item_details = item_details + "<br>" + explain_data[explain_idx]

# Get enchantment data.
enchant_text = ""
enchant_data = equipment_data[13]["enchant"]["status"]

# Loop thru every enchantment and get enchantment name and value.
for enc_idx, enc in enumerate(enchant_data):
    if enchant_text == "":
        enchant_text = enchant_data[enc_idx]["name"] + "+" + str(enchant_data[enc_idx]["value"])
    else:
        enchant_text = enchant_text + "<br>" + enchant_data[enc_idx]["name"] + "+" + str(enchant_data[enc_idx]["value"])

# Add enchant_text to item_details
item_details = item_details + "<br><br>&lt마법부여&gt<br>" + enchant_text

# Combine all information of one equipment into html_div_item_details and append to html_final_item_details.
html_final_item_details = html_final_item_details + html_div_item_details.format(item_img, reinforce_text, itemName_reinforce_text, "", enchant_text, item_details, "")

In [26]:
# Get equipment info for pet.

# Get title image.
item_img = item_img_std_link.format(equipment_data[14]["itemId"]) + \
            item_img_std_link.format(equipment_data[14]["clone"]["itemId"]) if "clone" in equipment_data[14] else ""

# Get reinforcement value.
reinforce_text = "+" + str(equipment_data[14]["reinforce"])

# Get item name and reinforcement values.
itemName_reinforce_text = equipment_data[14]["itemName"] + "<br>" + equipment_data[14]["clone"]["itemName"] if "clone" in equipment_data[14] else "" + \
                            "<br>피해증가 +" + str(equipment_data[14]["reinforce"])

# Get item details.
item_details = equipment_data[14]["itemRarity"] + " " + equipment_data[14]["itemTypeDetail"] + "<br>" + \
                str(equipment_data[14]["reinforce"]) + " 강화" + "<br>" + \
                "모험가 명성 " + str(equipment_data[14]["fame"]["total"]) + "<br>"

# (item details) get status data.
status_data = equipment_data[14]["itemStatus"]
# Loop thru every item status and get status name and value.
for status_idx, enc in enumerate(status_data):
    item_details = item_details + "<br>" + status_data[status_idx]["name"] + "+" + str(status_data[status_idx]["value"])

# Get clone details.
if "clone" in equipment_data[14]:
    clone_text = equipment_data[14]["clone"]["itemName"] + "<br>" + \
                    equipment_data[14]["clone"]["itemRarity"] + " " + equipment_data[14]["clone"]["itemTypeDetail"]
                    
# (clone details) get clone status data.
clone_status_data = equipment_data[14]["clone"]["itemStatus"]
# Loop thru every item status and get status name and value.
for clone_status_idx, clone_status in enumerate(clone_status_data):
    clone_text = clone_text + "<br>" + clone_status_data[clone_status_idx]["name"] + "+" + str(clone_status_data[clone_status_idx]["value"])


# Get skill enchantment data.
skillenc_text = ""
skillenc_data = equipment_data[14]["itemReinforceSkill"]

# Loop thru every job and get skill enchantment name and value.
for skillenc_idx, skillenc in enumerate(skillenc_data):
    # Loop thru every job skill.
    for skillenc_idx2, skillenc2 in enumerate(skillenc_data[skillenc_idx]["skills"]):
        if skillenc_text == "":
            skillenc_text = skillenc_data[skillenc_idx]["jobName"] + "<br>" + skillenc_data[skillenc_idx]["skills"][skillenc_idx2]["name"] + " +" + str(skillenc_data[skillenc_idx]["skills"][skillenc_idx2]["value"])
        else:
            skillenc_text = skillenc_text + "<br>" + skillenc_data[skillenc_idx]["jobName"] + "<br>" + skillenc_data[skillenc_idx]["skills"][skillenc_idx2]["name"] + " +" + str(skillenc_data[skillenc_idx]["skills"][skillenc_idx2]["value"])


# Add enchant_text to item_details
item_details = item_details + "<br><br>" + skillenc_text

# Combine all information of one equipment into html_div_item_details and append to html_final_item_details.
html_final_item_details = html_final_item_details + html_div_item_details.format(item_img, reinforce_text, itemName_reinforce_text, "", skillenc_text, item_details, clone_text)

In [27]:
# Get equipment info for insignia.

# Get insignia image.
item_img = item_img_std_link.format(equipment_data[15]["itemId"])

# Get reinforcement value.
reinforce_text = "+" + str(equipment_data[15]["reinforce"])

# Get item name.
itemName_text = equipment_data[15]["itemName"]

# get item details.
item_details = equipment_data[15]["itemRarity"] + " " + equipment_data[15]["itemTypeDetail"] + "<br>" + \
                "레벨 제한 " + str(equipment_data[15]["itemAvailableLevel"]) + "<br>" + \
                str(equipment_data[15]["reinforce"]) + " 강화" + "<br>" + \
                "모험가 명성 " + str(equipment_data[15]["fame"]["total"]) + "<br>"

# (item details) get status data.
status_data = equipment_data[15]["itemStatus"]
# Loop thru every item status and get status name and value.
for status_idx, enc in enumerate(status_data[:-1]):
    item_details = item_details + "<br>" + status_data[status_idx]["name"] + "+" + str(status_data[status_idx]["value"])


# Get gem data.
gem_text = ""
gem_data = equipment_data[15]["gems"]

# Loop thru every gem and get gem name.
for gem_idx, gem in enumerate(gem_data):
    if gem_text == "":
        gem_text = gem_data[gem_idx]["itemName"]
    else:
        gem_text = gem_text + "<br>" + gem_data[gem_idx]["itemName"]

# Add enchant_text to item_details
item_details = item_details + "<br><br>" + gem_text


# Combine all information of one equipment into html_div_item_details and append to html_final_item_details.
html_final_item_details = html_final_item_details + html_div_item_details.format(item_img, reinforce_text, itemName_text, "", gem_text, item_details, "")

In [30]:
# Create html file.
file_html = open("CharacterEqs_cain_ec362ecd3b3d2d9f213b5ea29dc72ab6.html", "w", encoding="utf-8")

# Write content into html file.
file_html.write(html_style + "\n" + html_final_item_details)

# Close html file
file_html.close()